Yoginii Waykole  
AIML

Bajaj Challenge!

In [ ]:
import json
import pandas as pd
from datetime import datetime

In [1]:
# Load the JSON data
file_path = '/content/DataEngineeringQ2.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Extract patient details
patient_details = [entry['patientDetails'] for entry in data]

# Create DataFrame
df = pd.DataFrame(patient_details)

# Define the columns to check for missing values
columns_to_check = ['firstName', 'lastName', 'birthDate']

# Calculate percentage of missing values for each column
missing_percentages = df[columns_to_check].isnull().mean() * 100
missing_percentages = missing_percentages.round(2)

# Add check for empty strings
for col in columns_to_check:
    missing_percentages[col] += (df[col] == '').mean() * 100

# Print the results
print(missing_percentages)


firstName     0.000000
lastName     70.967742
birthDate    32.260000
dtype: float64


In [2]:
# Extract patient details
patient_details = [entry['patientDetails'] for entry in data]

# Create DataFrame
df = pd.DataFrame(patient_details)

# Define the column to impute
gender_column = 'gender'

# Impute missing values with mode
mode_value = df[gender_column].mode()[0]
df[gender_column].replace('', None, inplace=True)  # Treat empty strings as missing values
df[gender_column].fillna(mode_value, inplace=True)

# Calculate percentage of female gender
female_percentage = (df[gender_column] == 'F').mean() * 100
female_percentage = round(female_percentage, 2)

# Print the result
print(f"Percentage of female gender after imputation: {female_percentage}%")


Percentage of female gender after imputation: 32.26%


In [10]:
from datetime import datetime

# Define the age groups
def determine_age_group(birth_date):
    if not birth_date:
        return None

    birth_date = datetime.strptime(birth_date, "%Y-%m-%dT%H:%M:%S.%fZ")
    age = (datetime.now() - birth_date).days // 365

    if age <= 12:
        return 'Child'
    elif 13 <= age <= 19:
        return 'Teen'
    elif 20 <= age <= 59:
        return 'Adult'
    elif age >= 60:
        return 'Senior'

# Add the age group column and count the number of Adults
adult_count = 0
for entry in data:
    birth_date = entry.get("patientDetails", {}).get("birthDate")
    age_group = determine_age_group(birth_date)
    if age_group:
        entry["ageGroup"] = age_group
        if age_group == 'Adult':
            adult_count += 1
    else:
        entry["ageGroup"] = "Unknown"

print(f'The count of patients in the "Adult" age group is: {adult_count}')

# Optionally, save the updated data back to a file
with open('/content/UpdatedDataEngineeringQ2.json', 'w') as file:
    json.dump(data, file, indent=4)

The count of patients in the "Adult" age group is: 21


In [16]:
import pandas as pd
import numpy as np
import json
from datetime import datetime

# Load JSON file
file_path = '/content/DataEngineeringQ2.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Print the structure of the JSON data
print(json.dumps(data, indent=4))

# Normalize JSON data to flat table
df = pd.json_normalize(
    data,
    record_path=['consultationData', 'medicines'],
    meta=[
        'appointmentId',
        ['patientDetails', 'firstName'],
        ['patientDetails', 'lastName'],
        ['patientDetails', 'birthDate'],
        ['patientDetails', 'gender']
    ],
    errors='ignore'
)

# Rename columns for convenience
df.columns = [col.split('.')[-1] for col in df.columns]

# Handling Missing Values
df['birthDate'] = pd.to_datetime(df['birthDate'], errors='coerce')
missing_values = df[['firstName', 'lastName', 'birthDate']].isnull() | (df[['firstName', 'lastName', 'birthDate']] == '')

# Percentage of Missing Values
missing_percentage = missing_values.mean() * 100
missing_percentage = missing_percentage.round(2)
print(f"Missing Percentage:\n{missing_percentage}\n")

# Impute gender with mode
gender_mode = df['gender'].mode()
if not gender_mode.empty:
    gender_mode = gender_mode[0]
    df['gender'].replace('', gender_mode, inplace=True)
    df['gender'].fillna(gender_mode, inplace=True)
else:
    gender_mode = 'Unknown'  # Handle case if no mode is found
female_percentage = (df['gender'] == 'F').mean() * 100
female_percentage = round(female_percentage, 2)
print(f"Percentage of Female Gender after Imputation: {female_percentage}%\n")

# Age Group Categorization
def age_group(birth_date):
    if pd.isna(birth_date):
        return 'Unknown'
    age = (datetime.now() - birth_date).days // 365
    if age <= 12:
        return 'Child'
    elif 13 <= age <= 19:
        return 'Teen'
    elif 20 <= age <= 59:
        return 'Adult'
    else:
        return 'Senior'

df['ageGroup'] = df['birthDate'].apply(age_group)
adult_count = (df['ageGroup'] == 'Adult').sum()
print(f"Count of Adult Patients: {adult_count}\n")

# Average Number of Medicines Prescribed
average_medicines = df.groupby('appointmentId')['medicineId'].nunique().mean()
average_medicines = round(average_medicines, 2)
print(f"Average Number of Medicines Prescribed: {average_medicines}\n")

# Third Most Frequently Prescribed Medicine
third_most_frequent_medicine = df['medicineName'].value_counts().index[2]
print(f"Third Most Frequently Prescribed Medicine: {third_most_frequent_medicine}\n")

# Percentage Distribution of Active and Inactive Medicines
active_percentage = (df['isActive'] == True).mean() * 100
inactive_percentage = (df['isActive'] == False).mean() * 100
active_percentage = round(active_percentage, 2)
inactive_percentage = round(inactive_percentage, 2)
print(f"Percentage Distribution of Active Medicines: {active_percentage}%")
print(f"Percentage Distribution of Inactive Medicines: {inactive_percentage}%\n")

# Save to a file
df.to_csv('./output_analysis.csv', index=False)

[
    {
        "_id": "T6hf3rb5",
        "appointmentId": "40d2-9c9f",
        "patientDetails": {
            "_id": "T6hb630b3",
            "firstName": "Css",
            "lastName": "",
            "emailId": "",
            "gender": "",
            "alternateContact": "",
            "birthDate": null
        },
        "phoneNumber": "96686896670",
        "consultationData": {
            "adviceTemplates": [],
            "advices": [],
            "attachments": [],
            "chiefComplaints": [],
            "customOne": [],
            "customThree": [],
            "customTwo": [],
            "disease": [],
            "doctorNotes": "",
            "emergencyInstructions": [],
            "emergencyInstructionsTemplate": [],
            "emrTemplates": [],
            "examinationNote": [],
            "findings": [],
            "investigationInstructions": [],
            "investigationTemplates": [],
            "investigations": [],
            "isBalicAppointm

In [20]:
import json

# Load the JSON file
file_path = '/content/DataEngineeringQ2.json'
with open(file_path, 'r') as file:
    data = json.load(file)

def is_valid_indian_phone_number(phone_number):
    # Remove prefix if exists
    if phone_number.startswith('+91'):
        phone_number = phone_number[3:]
    elif phone_number.startswith('91'):
        phone_number = phone_number[2:]

    # Check if the remaining part is a 10-digit number
    if len(phone_number) == 10 and phone_number.isdigit():
        number = int(phone_number)
        return 6000000000 <= number <= 9999999999
    return False

# Add the isValidMobile column and count valid phone numbers
valid_phone_count = 0
for entry in data:
    phone_number = entry.get("phoneNumber", "")
    is_valid = is_valid_indian_phone_number(phone_number)
    entry["isValidMobile"] = is_valid
    if is_valid:
        valid_phone_count += 1

print(f'The number of valid phone numbers is: {valid_phone_count}')

The number of valid phone numbers is: 18
